In [1]:
import nutils
import pandas as pd
import numpy as np
from common import *

In [2]:
# import os
# for code in SELECTED_CODES:
#     if os.path.exists(f"A:\snapshot\stkCode_{code}.parquet"):
#         continue
#     datas = get_snapshot(code)
#     df = pd.DataFrame(datas['tickData'],columns=COLS_SNAPSHOTS)
#     df['date'] = datas['timestamp'][:,0]
#     df['time'] = datas['timestamp'][:,1]
#     df.set_index('date',inplace=True)
#     df.set_index('time',inplace=True,append=True)
#     df.to_parquet(f"A:\snapshot\stkCode_{code}.parquet",)

In [3]:
code = "000537"
datas = get_snapshot(code)

In [4]:
datas

{'tickData': array([[6.270e+00, 6.260e+00, 6.260e+00, ..., 5.600e+03, 3.000e+03,
         1.000e+04],
        [6.270e+00, 6.260e+00, 6.260e+00, ..., 1.200e+04, 8.500e+03,
         1.380e+04],
        [6.270e+00, 6.260e+00, 6.260e+00, ..., 1.200e+04, 8.500e+03,
         1.380e+04],
        ...,
        [1.327e+01, 1.333e+01, 1.364e+01, ..., 1.000e+03, 5.500e+03,
         9.000e+03],
        [1.327e+01, 1.333e+01, 1.364e+01, ..., 1.000e+03, 5.500e+03,
         9.000e+03],
        [1.327e+01, 1.333e+01, 1.364e+01, ..., 5.500e+03, 9.000e+03,
         1.300e+03]]),
 'timestamp': array([[20210104,    93000],
        [20210104,    93003],
        [20210104,    93006],
        ...,
        [20221230,   145651],
        [20221230,   145654],
        [20221230,   145657]])}

In [ ]:
import os


def gen_sa(code):
    datas = get_snapshot(code)
    df = pd.DataFrame(datas["tickData"], columns=COLS_SNAPSHOTS)
    # df['date'] = datas['timestamp'][:,0]
    # df['time'] = datas['timestamp'][:,1]
    # shape_dict[code] = df.shape

    snapshot = np.memmap(
        f"A:/sa/snapshot/{code}.dat", dtype=np.float32, mode="w+", shape=df.shape
    )
    snapshot[:] = df.values
    snapshot.flush()
    snapshot.flags["WRITEABLE"] = False

    tmstamp = np.memmap(
        f"A:/sa/timestamp/{code}.dat",
        dtype=np.int64,
        mode="w+",
        shape=datas["timestamp"].shape,
    )
    tmstamp[:] = datas["timestamp"]
    tmstamp.flush()
    tmstamp.flags["WRITEABLE"] = False

    assert len(snapshot) == len(tmstamp), f"{len(snapshot)}!= {len(tmstamp)}"


os.makedirs("A:/sa/snapshot", exist_ok=True)
os.makedirs("A:/sa/timestamp", exist_ok=True)
for code in SELECTED_CODES:
    if os.path.exists(f"A:\snapshot\{code}.dat"):
        continue
    else:
        gen_sa(code)

In [ ]:
print(tmstamp.shape)

In [ ]:
def gen_sa_ret(code):
    datas = get_snapshot(code)
    df = pd.DataFrame(datas["tickData"], columns=COLS_SNAPSHOTS)
    df["mid_price"] = (df["BidPrice1"] + df["AskPrice1"]) / 2
    df["ret"] = np.log(df.mid_price.shift(-60) / df.mid_price) * 1e4

    fp = np.memmap(
        f"A:/sa/ret/{code}.dat",
        dtype=np.float32,
        mode="w+",
        shape=df["ret"].shape,
    )
    fp[:] = df["ret"]
    fp.flush()
    fp.flags["WRITEABLE"] = False
    
for code in tqdm(SELECTED_CODES):
    gen_sa_ret(code)

In [ ]:
df = np.memmap(
    'A:/sa/snapshot/000537,npy',
    dtype='float32',
    mode='r',
)

In [ ]:
df = df.reshape(-1,len(COLS_SNAPSHOTS))

In [ ]:
datas = get_snapshot("000537")
df = pd.DataFrame(datas['tickData'],columns=COLS_SNAPSHOTS)

In [ ]:
df.shape